In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon

Mounted at /content/drive
/content/drive/MyDrive/Github/MachineLearning/Module_3_Bank_Proj/Brandon


In [7]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv")

In [8]:
#Profiling of Dataset

#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 
from pandas_profiling import ProfileReport as pr
profile = pr(data)
profile.to_notebook_iframe()

OSError: ignored

In [ ]:
### Geo Location Pre-Processing ###

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5272200045,20141113T000000,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,378000.0
19996,9578500790,20141111T000000,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,399950.0
19997,7202350480,20140930T000000,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,575000.0
19998,1723049033,20140620T000000,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,245000.0


In [9]:
import altair as alt

from altair import pipe, limit_rows, to_values
t = lambda data: pipe(data, limit_rows(max_rows=20000), to_values)
alt.data_transformers.register('custom', t)
alt.data_transformers.enable('custom')

data = data[data.lat > 45]
data = data[data.long < -110]

data_high = data[data.price > 500000]
data_low = data[data.price <= 500000]

high_chart = alt.Chart(data_high).mark_circle().encode(
    alt.X("long:Q",
          scale=alt.Scale(zero = False)),
    alt.Y("lat:Q",
          scale=alt.Scale(zero = False)),
          color = "price"
)

low_chart = alt.Chart(data_low).mark_circle().encode(
    alt.X("long:Q",
          scale=alt.Scale(zero = False)),
    alt.Y("lat:Q",
          scale=alt.Scale(zero = False)),
          color = "price"
)

high_chart


Output hidden; open in https://colab.research.google.com to view.

In [10]:
low_chart

Output hidden; open in https://colab.research.google.com to view.

In [27]:
alt.Chart(data).mark_rect().encode(
    alt.X('long:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('lat:Q', bin=alt.Bin(maxbins=100)),
    alt.Color('mean(price):Q', scale=alt.Scale(scheme='greenblue'))
)

Output hidden; open in https://colab.research.google.com to view.

In [35]:
alt.Chart(data).mark_rect().encode(
    alt.X('long:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('lat:Q', bin=alt.Bin(maxbins=100)),
    alt.Color('count(id):Q', scale=alt.Scale(scheme='greenblue'))
)

Output hidden; open in https://colab.research.google.com to view.

In [29]:
data['lat_bin'] = 0
data['long_bin'] = 0
for lat in range(4715,4779,1):
  if lat + 1 > 4779:
    break
  else:
    data.loc[(data.lat >= (lat/100)) & (data.lat < ((lat/100) + .01)),'lat_bin'] = lat/100
  
for long in range(-12252,-12124,2):
  if long + 1 > -12124:
    break
  else:
    data.loc[(data.long >= (long/100)) & (data.long < ((long/100) + .02)),'long_bin'] = long/100
  

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,lat_bin,long_bin
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,8,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,47.38,-122.06
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,7,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,47.30,-122.40
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,7,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,47.56,-122.40
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,8,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,47.31,-122.40
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,7,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,47.68,-122.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5272200045,20141113T000000,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,1947,0,98125,47.7144,-122.319,1000,6947,378000.0,47.71,-122.32
19996,9578500790,20141111T000000,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,2014,0,98023,47.2974,-122.349,2927,5183,399950.0,47.29,-122.36
19997,7202350480,20140930T000000,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,2004,0,98053,47.6810,-122.032,1690,2650,575000.0,47.68,-122.04
19998,1723049033,20140620T000000,1,0.75,380,15000,1.0,0,0,3,5,380,0,1963,0,98168,47.4810,-122.323,1170,15000,245000.0,47.48,-122.34


In [50]:
(data.groupby(["lat_bin","long_bin"], as_index=False)
  .agg({"price":"mean", "id":"count"})
  .rename(columns = {"price":"avg_neighborhood_price", "id":"no_houses_neighborhood"}))


,lat_bin,long_bin,avg_neighborhood_price,no_houses_neighborhood
0,47.15,-121.96,687000.000000,1
1,47.15,-121.66,380000.000000,1
2,47.16,-121.98,542500.000000,2
3,47.17,-122.04,360000.000000,1
4,47.17,-122.02,230000.000000,1
...,...,...,...,...
1181,47.77,-122.06,849333.333333,3
1182,47.77,-122.04,603750.000000,2
1183,47.77,-121.94,637000.000000,1
1184,47.77,-121.92,309637.500000,2


In [80]:
test = data.groupby(["lat_bin", "long_bin"], group_keys = False).apply(lambda x: np.mean(x.price))

test

lat_bin  long_bin
47.15    -121.96     687000.000000
         -121.66     380000.000000
47.16    -121.98     542500.000000
47.17    -122.04     360000.000000
         -122.02     230000.000000
                         ...      
47.77    -122.06     849333.333333
         -122.04     603750.000000
         -121.94     637000.000000
         -121.92     309637.500000
         -121.90     420600.000000
Length: 1186, dtype: float64

AttributeError: ignored